# Economic & Health Benefits Model

### Jupyter Notebook
This notebook provides a hypothetical model estimating the potential economic and health benefits of an intervention that reduces fat mass by 2 lb and increases muscle mass by 2 lb.

## 1. Introduction
In this illustrative scenario, we assume:
- The intervention costs $500 per person per year.
- It reduces obesity-related healthcare costs by a modest percentage (5% to 10%) of obesity-attributable expenses.
- We also consider Quality-Adjusted Life Year (QALY) improvements.

We'll calculate per-person net cost savings (or cost increases) and then scale it up to Medicare (≈65 million beneficiaries) and the total U.S. population (≈330 million), assuming a 50% adoption rate. Finally, we include a basic sensitivity analysis.


## 2. Parameter Definitions
Below are the main parameters used in the model:

| Parameter                                  | Symbol         | Default Value          | Notes                                             |
|:-----------------------------------------:|:--------------:|:----------------------:|:--------------------------------------------------|
| Average annual total healthcare cost      | C_base         | $6,000                | Baseline yearly healthcare spending per adult     |
| Fraction of costs related to obesity      | p_ob           | 0.20 (20%)            | Portion of C_base linked to overweight/obesity   |
| Relative reduction in obesity-related costs | δ            | Range (0.05 - 0.10)   | Fractional decrease in obesity costs             |
| Cost of therapy (annual)                  | C_therapy      | $500                  | Intervention cost per person per year            |
| QALY improvement per year                 | ΔQ            | 0.01 QALYs/yr         | Incremental gain in quality of life              |
| Value of one QALY                         | V_QALY         | $100,000              | Conventional midpoint for cost-effectiveness     |
| Medicare beneficiaries (approx.)          | N_M            | 65 million            | For population-level estimates                   |
| US population (approx.)                   | N_US           | 330 million           | For population-level estimates                   |
| Adoption rate                             | ρ             | 0.50 (50%)            | Fraction of target population receiving therapy  |
| Discount rate (for multi-year analysis)   | r              | 3% (0.03)             | Annual discount rate for Net Present Value       |
| Time horizon                              | T              | 5 years               | Example duration to evaluate cost savings        |


### Accounting for QALYs

If we factor in QALYs (Quality-Adjusted Life Years), the annual value gained per person is:

V_QALY_annual = ΔQ × V_QALY

We'll add this amount to the direct cost offset each year.

## 3. Population-Level Estimates
Assume:
- N_M = 65 million Medicare beneficiaries
- N_US = 330 million total population
- ρ = 50% adoption rate


### Discussion
- Modifying assumptions about δ, C_therapy, or ΔQ drastically changes whether the intervention is cost-saving or cost-effective.
V_QALY = 100000.0       # Value of one QALY
N_M = 65e6              # Medicare population
N_US = 330e6            # Total US population
rho = 0.50              # Adoption rate
r = 0.03                # Discount rate
T = 5                   # Time horizon in years

# Compute baseline obesity-related cost
C_obesity = C_base * p_ob
print(f"Obesity-related annual healthcare cost: ${C_obesity:,.2f}")

# Function to compute annual net cost savings (without QALYs) per person
def annual_net_savings(delta, C_obesity, C_therapy):
    # delta = fraction reduction in obesity-related costs
    cost_savings = C_obesity * delta   # annual cost savings from the intervention
    net_savings = cost_savings - C_therapy
    return net_savings

# Compute for both low and high delta
net_savings_low = annual_net_savings(delta_low, C_obesity, C_therapy)
net_savings_high = annual_net_savings(delta_high, C_obesity, C_therapy)

print(f"\nIf delta = {delta_low:.0%} => Net annual savings per person: ${net_savings_low:,.2f}")
print(f"If delta = {delta_high:.0%} => Net annual savings per person: ${net_savings_high:,.2f}")


### Multi-Year Net Present Value (NPV)
We can compute a simplified NPV of the annual net cash flows over 5 years.


In [1]:
def npv_of_savings(net_savings, r, T):
    # net_savings = net savings each year (constant)
    # r = discount rate
    # T = time horizon in years
    npv = 0.0
    for t in range(1, T+1):
        npv += net_savings / ((1 + r)**t)
    return npv

npv_low = npv_of_savings(net_savings_low, r, T)
npv_high = npv_of_savings(net_savings_high, r, T)

print(f"NPV over {T} years (delta={delta_low:.0%}): ${npv_low:,.2f}")
print(f"NPV over {T} years (delta={delta_high:.0%}): ${npv_high:,.2f}")


NameError: name 'net_savings_low' is not defined

### Accounting for QALYs

If we factor in QALYs (Quality-Adjusted Life Years), the annual value gained per person is:
\[
V_{\text{QALY, annual}} = \Delta Q \times V_{\text{QALY}}
\]
We'll add this amount to the direct cost offset each year.

In [7]:
def annual_net_with_qalys(delta, C_obesity, C_therapy, Delta_Q, V_QALY):
    cost_savings = C_obesity * delta
    qalys_value = Delta_Q * V_QALY
    return cost_savings - C_therapy + qalys_value

net_with_qalys_low = annual_net_with_qalys(delta_low, C_obesity, C_therapy, Delta_Q, V_QALY)
net_with_qalys_high = annual_net_with_qalys(delta_high, C_obesity, C_therapy, Delta_Q, V_QALY)

print(f"Annual net benefit (delta={delta_low:.0%}) including QALYs: ${net_with_qalys_low:,.2f}")
print(f"Annual net benefit (delta={delta_high:.0%}) including QALYs: ${net_with_qalys_high:,.2f}")

# Compute NPV of the scenario that includes QALY benefits
npv_with_qalys_low = npv_of_savings(net_with_qalys_low, r, T)
npv_with_qalys_high = npv_of_savings(net_with_qalys_high, r, T)

print(f"\nNPV over {T} years (delta={delta_low:.0%}, with QALYs): ${npv_with_qalys_low:,.2f}")
print(f"NPV over {T} years (delta={delta_high:.0%}, with QALYs): ${npv_with_qalys_high:,.2f}")

Annual net benefit (delta=5%) including QALYs: $560.00
Annual net benefit (delta=10%) including QALYs: $620.00

NPV over 5 years (delta=5%, with QALYs): $2,564.64
NPV over 5 years (delta=10%, with QALYs): $2,839.42


## 3. Population-Level Estimates
Assume:
- \(N_M = 65\) million Medicare beneficiaries
- \(N_{US} = 330\) million total population
- \(\rho = 50\%\) adoption rate


In [8]:
# First, define a helper to compute total population-level net benefit
def total_annual_benefit(net_savings_per_person, population, adoption_rate):
    return net_savings_per_person * population * adoption_rate

# Direct cost offset only (delta=0.10 for example)
annual_savings_pop = total_annual_benefit(net_savings_high, N_M, rho)
print(f"Medicare Population (delta=10%, 50% adoption), Annual Direct Offset Only: ${annual_savings_pop/1e9:,.2f}B")

# Now including QALYs
annual_with_qalys_pop = total_annual_benefit(net_with_qalys_high, N_M, rho)
print(f"Medicare Population (delta=10%, 50% adoption), Annual w/ QALYs: ${annual_with_qalys_pop/1e9:,.2f}B")

# Similarly for total US population
annual_savings_pop_us = total_annual_benefit(net_savings_high, N_US, rho)
annual_with_qalys_pop_us = total_annual_benefit(net_with_qalys_high, N_US, rho)

print(f"\nUS Population (delta=10%, 50% adoption), Annual Direct Offset Only: ${annual_savings_pop_us/1e9:,.2f}B")
print(f"US Population (delta=10%, 50% adoption), Annual w/ QALYs: ${annual_with_qalys_pop_us/1e9:,.2f}B")

Medicare Population (delta=10%, 50% adoption), Annual Direct Offset Only: $-12.35B
Medicare Population (delta=10%, 50% adoption), Annual w/ QALYs: $20.15B

US Population (delta=10%, 50% adoption), Annual Direct Offset Only: $-62.70B
US Population (delta=10%, 50% adoption), Annual w/ QALYs: $102.30B


## 4. Sensitivity Analysis
Because results can vary dramatically with small changes in assumptions, it's helpful to loop over different values of \(\delta\) and therapy cost, etc. Here is a simple example.

In [9]:
delta_values = [0.05, 0.10, 0.15, 0.20]
therapy_costs = [200, 300, 500, 800]

print("delta\tC_therapy\tNet_Savings\tNet_w_QALYs")
for d in delta_values:
    for c in therapy_costs:
        ns = (C_obesity * d) - c
        ns_q = (C_obesity * d) - c + (Delta_Q * V_QALY)
        print(f"{d*100:.0f}%\t${c}\t\t${ns:,.2f}\t\t${ns_q:,.2f}")

delta	C_therapy	Net_Savings	Net_w_QALYs
5%	$200		$-140.00		$860.00
5%	$300		$-240.00		$760.00
5%	$500		$-440.00		$560.00
5%	$800		$-740.00		$260.00
10%	$200		$-80.00		$920.00
10%	$300		$-180.00		$820.00
10%	$500		$-380.00		$620.00
10%	$800		$-680.00		$320.00
15%	$200		$-20.00		$980.00
15%	$300		$-120.00		$880.00
15%	$500		$-320.00		$680.00
15%	$800		$-620.00		$380.00
20%	$200		$40.00		$1,040.00
20%	$300		$-60.00		$940.00
20%	$500		$-260.00		$740.00
20%	$800		$-560.00		$440.00


### Discussion
- Modifying assumptions about \(\delta\), \(C_{therapy}\), or \(\Delta Q\) drastically changes whether the intervention is cost-saving or cost-effective.
- In real-world policy analysis, you'd also consider the **longer-term** impact on chronic disease incidence (like cardiovascular events, diabetes complications, etc.), which might yield **larger** cost savings than this simple 5%–10% assumption.
- For coverage decisions in Medicare, factoring in QALYs often makes an intervention more favorable if the improvements in quality of life are consistent and well-documented.


## 5. Conclusion
This notebook demonstrates a simplified approach to evaluating the cost and health impact of an intervention aimed at improving body composition by \(-2 lb\) fat and \(+2 lb\) muscle. **Real-world data** would be needed to calibrate the parameters for more accurate projections of healthcare cost reductions and QALY improvements.

---
**Next Steps**:
1. Conduct or reference **empirical studies** measuring changes in obesity-related outcomes for individuals receiving the intervention.
2. Gather **longitudinal data** (5- to 10-year disease incidence) to refine discounting and time-horizon assumptions.
3. Perform **robust sensitivity analyses** on therapy costs, adoption rates, and population-specific factors (e.g., age distribution, comorbidity prevalence).